In [1]:
import linguamind.linalg as la
import linguamind.nn as nn
import linguamind.nlp as nlp

from collections import Counter
import random

path ='/Users/amberedmundson/Laboratory/datasets/harryPotter/harryPotterAsciiCleaned.txt'
# path = '/Users/amberedmundson/Laboratory/clones/word2vec/text.txt'

v = nlp.Vocab()
t = nlp.Text(path,v)

v2 = nlp.Vocab(t.vocab)
v2.sort(10)

In [2]:
t.ChangeVocab(v2)
v2.InitUnigramTable()

vocab_size = v2.size
dim = 64

seed = la.Seed(1)

syn0 = nn.SparseLinearInput(vocab_size,dim)
syn0.weights.uniform(seed)
syn0.weights -= 0.5
syn0.weights /= dim

syn1neg = nn.SparseLinearOutput(dim,vocab_size)
syn1neg.weights.zero()

layers = list()
layers.append(syn0)
layers.append(syn1neg)
layers.append(nn.Sigmoid(vocab_size))

mlp = nn.Sequential(layers)
criterion = nn.MSECriterion()
optim = nn.StochasticGradient(mlp,criterion)

In [3]:
words = 0
for sent in t.sentences:
    for word in sent:
        words+=1

In [4]:
words

1381594

In [5]:
1421660

1421660

In [6]:
cbow = nn.CBOW(t.sentences,v2,10,10)

In [ ]:
%%timeit
optim.train(cbow,0.01,10,1)

In [ ]:
%%timeit
optim.train(cbow,0.01,10,4)

In [10]:
import numpy as np
word2vec = {}
vocab = list()
for i in range(v2.size):
    l2 = np.sqrt(syn0.weights[i].dot(syn0.weights[i]))
    v = syn0.weights[i]
    v /= l2
    vocab.append(v2.getTermAtIndex(i).letters)
    word2vec[v2.getTermAtIndex(i).letters] = syn0.weights[i]
    
sim = Counter()
for word in vocab:
    sim[word] = word2vec['hermione'].dot(word2vec[word])
sim.most_common()

[('hermione', 1.0),
 ('ron', 0.943290114402771),
 ('marietta', 0.8725757598876953),
 ('ginny', 0.863684356212616),
 ('george', 0.8603859543800354),
 ('anxious', 0.8530943393707275),
 ('staring', 0.8459852337837219),
 ('bending', 0.8450285792350769),
 ('pansy', 0.844501793384552),
 ('fred', 0.8384718894958496),
 ('looking', 0.8372832536697388),
 ('lupin', 0.8337599039077759),
 ('neville', 0.8296249508857727),
 ('sobbing', 0.8296082019805908),
 ('seamus', 0.8265498876571655),
 ('frozen', 0.8260033130645752),
 ('blinking', 0.8248494267463684),
 ('pointing', 0.8188552260398865),
 ('sleek', 0.8182573914527893),
 ('dreamy', 0.818143904209137),
 ('hagrid', 0.8148995041847229),
 ('settled', 0.8102484345436096),
 ('luna', 0.8093511462211609),
 ('dancing', 0.8086775541305542),
 ('dried', 0.8061267137527466),
 ('goyle', 0.8043704628944397),
 ('parvati', 0.8041970729827881),
 ('zabini', 0.80325847864151),
 ('pacing', 0.8032236695289612),
 ('stare', 0.7996407747268677),
 ('aghast', 0.79924976825714

# Forward Prop Modification for Windowing Layers

after you have confirmed that you are doing everything that word2vec does (exactly the same!!!)... such that you duplicate results, you should try this highly experimental thing

For CBOW, you should be able to tweak the output embedding based on how the inputs change and the previous gradient instead of regenerating it every time. If your window size is 5, this should give you close to a 5 * 2 = 10x speedup for the input window generation (which is where most of the parameters are going anyway)